In [ ]:
import os
os.chdir("..")

import json
import pandas as pd
import ast
import tabulate

from tool.ner_metrics import *
from tool.gender_checker import get_personal_titles
from tool.annotations_utils import read_annotations, has_intersection, fix_personal_titles, personal_titles_stats

In [ ]:
gold_standard_path = "data/testing_sets/test_person_gold_standard_corrected"
titles = os.listdir(gold_standard_path)
annotations_person = read_annotations(os.path.join(gold_standard_path, titles[0]))

prediction_path = "experiments/ner_fixed/flair__ner-large/"
predictions_person = read_annotations(os.path.join(prediction_path, titles[0]))

In [ ]:
entities_gold, sentences = data_from_json(os.path.join(gold_standard_path, titles[0]))
entities_matcher, _ = data_from_json(os.path.join(prediction_path, titles[0]))
entities_gold = [[list(x) for x in set(tuple(x) for x in sent_gold_entities)] for sent_gold_entities in
                 entities_gold]
gold, matcher, errors = organize_entities(entities_gold, entities_matcher, sentences, True)

In [ ]:
def get_errors_stats(gold_path, pred_path):
    results = []
    for title in os.listdir(os.path.join(gold_path)):
        gold_annotations = read_annotations(os.path.join(gold_path, title))
        pred_annotations = read_annotations(os.path.join(pred_path, title))
        
        gold_count = 0
        pred_count = 0
        exact_count = 0
        wrong_hero_count = 0
        intersections_count = 0
        incorrect_count = 0
        missing_count = 0

        for gold, pred in zip(gold_annotations, pred_annotations):
            gold_entities = gold['entities']
            pred_entities = pred['entities']

            gold_count += len(gold_entities)
            pred_count += len(pred_entities)

            matched_count = 0
            for entity1 in gold_entities:
                exact = False
                intersection = False

                for entity2 in pred_entities:
                    if entity1 == entity2:
                        exact_count += 1
                        exact = True
                        matched_count += 1
                        break

                if not exact:
                    for entity2 in pred_entities:
                        if has_intersection(entity1, entity2):
                            intersections_count += 1
                            intersection = True
                            matched_count += 1
                            break

                if not exact and not intersection:
                    missing_count += 1

            incorrect_count += len(pred_entities) - matched_count

            assert gold_count + incorrect_count == pred_count + missing_count
            assert gold_count - exact_count - intersections_count - wrong_hero_count \
                   == missing_count
        
        title_results = {'Title': title.split('.')[0].replace('_', ' '), 
                         '# goldstandard': gold_count, 
                         '# prediction': pred_count, 
                         '# incorrect annotations': incorrect_count,
                         '# annotations with wrong boundaries': intersections_count, 
                         '# missing annotations':  missing_count}
        results.append(title_results)
    return pd.DataFrame(results)

In [ ]:
df = get_errors_stats(gold_standard_path, prediction_path)

In [ ]:
print(tabulate.tabulate(df, headers = df.columns, tablefmt='latex_booktabs'))